## SORT

In [1]:
device = "CUDAExecutionProvider" # CPUExecutionProvider CUDAExecutionProvider
if device == 'CPUExecutionProvider':
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
import numpy as np
from PIL import Image, ImageDraw, ImageColor
import math
import onnxruntime as rt
import time
import importlib
import os
import cv2
from tqdm import notebook
DETRAC_TEST = '/home/ubuntu/ibrahim/master/datasets/Insight-MVT_Annotation_Test'
th = 0.5
os.chdir(os.path.abspath('/home/ubuntu/ibrahim/master/models/research/'))

In [3]:
directory="MVI_39031"
def calc_fps(det_elapsed: list, trac_elapsed: list):
    det_mean = sum(det_elapsed) / float(len(det_elapsed))
    trac_mean = sum(trac_elapsed) / float(len(trac_elapsed))
    total = list(map(sum, zip(det_elapsed, trac_elapsed)))
    fps_mean = sum(total) / float(len(total))
    print('Elapsed time: : ' + str(det_mean*1000) + ' millisecond per image')
    print('Elapsed time: : ' + str(trac_mean*1000) + ' millisecond per image')
    print("FPS: ", 1/fps_mean)

In [6]:
import research.object_detection.tracking.sort.sort
from research.object_detection.tracking.sort.sort import Sort

model_path = "/home/ubuntu/ibrahim/master/training/ssd_mobilenet/base/saved_model/"
#model_path = "/home/ubuntu/ibrahim/master/training/ssd_resnet/base/saved_model/"
drop = 2

sess = rt.InferenceSession(os.path.join(model_path, "model.onnx"), providers=[device])
outputs = ["num_detections", "detection_boxes", "detection_scores", "detection_classes"]

det_elapsed = []
trac_elapsed = []
importlib.reload(research.object_detection.tracking.sort.sort)
mot_tracker= Sort()
i=0
for image in notebook.tqdm(sorted(os.listdir(os.path.join(DETRAC_TEST, directory)))):
    i+=1
    if i%drop != 0:
        det_elapsed.append(0)
        trac_elapsed.append(0)
        continue
    image_path = os.path.join(DETRAC_TEST, directory, image)
    img = Image.open(image_path)
    image_np = np.asarray(img).reshape(img.size[1], img.size[0], 3)
    height, width, _ = image_np.shape
    input_tensor = np.expand_dims(image_np.astype(np.uint8), axis=0)
    det_start_time = time.time()
    result = sess.run(outputs, {"input_tensor": input_tensor})
    det_elapsed.append(time.time() - det_start_time)
    num_detections, boxes, scores, classes = result[0][0], result[1][0], result[2][0], result[3][0]
    det = []
    for f in range(boxes.shape[0]):
        box = boxes[f]
        if scores[f] > th:
            box_top = int(box[0] * height)
            box_left = int(box[1] * width)
            box_bottom = int(box[2] * height)
            box_right = int(box[3] * width)
            det.append([box_left, box_top, box_right, box_bottom, scores[f], classes[f]])
    
    trac_start_time = time.time()
    if det:
        track_bbs_ids = mot_tracker.update(np.array(det))
    else:
        track_bbs_ids = mot_tracker.update()
    trac_elapsed.append(time.time() - trac_start_time)
calc_fps(det_elapsed, trac_elapsed)

  0%|          | 0/1470 [00:00<?, ?it/s]

Elapsed time: : 8.964769045511881 millisecond per image
Elapsed time: : 0.7976001622725506 millisecond per image
FPS:  102.434150841438


# DeepSORT

In [4]:
from research.object_detection.tracking.deep_sort.deep_sort import nn_matching
from research.object_detection.tracking.deep_sort.deep_sort.detection import Detection
import research.object_detection.tracking.deep_sort.deep_sort.tracker
from research.object_detection.tracking.deep_sort.deep_sort.tracker import Tracker
from research.object_detection.tracking.deep_sort.tools import generate_detections as gdet
model_filename_deep= '/home/ubuntu/ibrahim/master/deep_sort/cosine_metric_learning/output/detrac/cosine-softmax/detrac.pb'
encoder = gdet.create_box_encoder(model_filename_deep,batch_size=1)

2022-02-21 01:13:09.262533: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-21 01:13:09.937978: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10037 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1


In [5]:
model_path = "/home/ubuntu/ibrahim/master/training/ssd_resnet/base/saved_model/"

sess = rt.InferenceSession(os.path.join(model_path, "model.onnx"), providers=[device])
outputs = ["num_detections", "detection_boxes", "detection_scores", "detection_classes"]

det_elapsed = []
trac_elapsed = []
importlib.reload(research.object_detection.tracking.deep_sort.deep_sort.tracker)
deep_tracker = Tracker(nn_matching.NearestNeighborDistanceMetric("cosine", 0.3, None))
for image in notebook.tqdm(sorted(os.listdir(os.path.join(DETRAC_TEST, directory)))):
    image_path = os.path.join(DETRAC_TEST, directory, image)
    img = Image.open(image_path)
    image_np = np.asarray(img).reshape(img.size[1], img.size[0], 3)
    height, width, _ = image_np.shape
    input_tensor = np.expand_dims(image_np.astype(np.uint8), axis=0)
    det_start_time = time.time()
    result = sess.run(outputs, {"input_tensor": input_tensor})
    num_detections, boxes, scores, classes = result[0][0], result[1][0], result[2][0], result[3][0]
    det = []
    for f in range(boxes.shape[0]):
        box = boxes[f]
        if scores[f] > th:
            box_top = int(box[0] * height)
            box_left = int(box[1] * width)
            box_bottom = int(box[2] * height)
            box_right = int(box[3] * width)
            det.append([box_left, box_top, (box_right-box_left), (box_bottom-box_top), scores[f], classes[f]])
    if det:
         features = encoder(image_np, np.array(det)[:, 0:4])
    else:
        features = encoder(image_np, np.array(det))
    det_elapsed.append(time.time() - det_start_time)
    detections = [Detection((bbox[0], bbox[1], bbox[2], bbox[3]), bbox[4], feature)
                            for bbox, feature in zip(det, features)]
    trac_start_time = time.time()
    deep_tracker.predict()
    deep_tracker.update(detections)
    track_bbs_ids = []
    for track in deep_tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue
        track_bbs_ids.append([*track.to_tlbr(), track.track_id])
    for track_pred in track_bbs_ids:
        bb_w = track_pred[2] - track_pred[0]
        bb_h = track_pred[3] - track_pred[1]
    trac_elapsed.append(time.time() - trac_start_time)
calc_fps(det_elapsed, trac_elapsed)

  0%|          | 0/1470 [00:00<?, ?it/s]

2022-02-21 01:13:16.404608: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8302
2022-02-21 01:13:16.474682: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


Elapsed time: : 268.81930163117494 millisecond per image
Elapsed time: : 6.126545562225134 millisecond per image
FPS:  3.63707984756936


# PAE

In [4]:
from research.object_detection.tracking.deep_sort.deep_sort import nn_matching
from research.object_detection.tracking.deep_sort.deep_sort.detection import Detection
import research.object_detection.tracking.deep_sort.deep_sort.tracker
from research.object_detection.tracking.deep_sort.deep_sort.tracker import Tracker

In [5]:
#model_path = "/home/ubuntu/ibrahim/master/training/ssd_mobilenet/final_embeddings/saved_model/"
model_path = "/home/ubuntu/ibrahim/master/training/ssd_resnet/final_embeddings/saved_model/"

sess = rt.InferenceSession(os.path.join(model_path, "model.onnx"), providers=[device])
outputs = ["num_detections", "detection_boxes", "detection_scores", "detection_classes", "nms_embedding"]

det_elapsed = []
trac_elapsed = []
importlib.reload(research.object_detection.tracking.deep_sort.deep_sort.tracker)
deep_tracker = Tracker(nn_matching.NearestNeighborDistanceMetric("cosine", 0.3, None))
for image in notebook.tqdm(sorted(os.listdir(os.path.join(DETRAC_TEST, directory)))):
    image_path = os.path.join(DETRAC_TEST, directory, image)
    img = Image.open(image_path)
    image_np = np.asarray(img).reshape(img.size[1], img.size[0], 3)
    height, width, _ = image_np.shape
    input_tensor = np.expand_dims(image_np.astype(np.uint8), axis=0)
    det_start_time = time.time()
    result = sess.run(outputs, {"input_tensor": input_tensor})
    det_elapsed.append(time.time() - det_start_time)
    num_detections, boxes, scores, classes, emb = result[0][0], result[1][0], result[2][0], result[3][0], result[4][0]
    det = []
    features = []
    for f in range(boxes.shape[0]):
        box = boxes[f]
        if scores[f] > th:
            box_top = int(box[0] * height)
            box_left = int(box[1] * width)
            box_bottom = int(box[2] * height)
            box_right = int(box[3] * width)
            det.append([box_left, box_top, (box_right-box_left), (box_bottom-box_top), scores[f], classes[f]])
            features.append(emb[f])

    detections = [Detection((bbox[0], bbox[1], bbox[2], bbox[3]), bbox[4], feature)
                            for bbox, feature in zip(det, features)]
    trac_start_time = time.time()
    deep_tracker.predict()
    deep_tracker.update(detections)
    track_bbs_ids = []
    for track in deep_tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue
        track_bbs_ids.append([*track.to_tlbr(), track.track_id])
    for track_pred in track_bbs_ids:
        bb_w = track_pred[2] - track_pred[0]
        bb_h = track_pred[3] - track_pred[1]
    trac_elapsed.append(time.time() - trac_start_time)
calc_fps(det_elapsed, trac_elapsed)

  0%|          | 0/1470 [00:00<?, ?it/s]

Elapsed time: : 252.04627140849627 millisecond per image
Elapsed time: : 4.293223627570535 millisecond per image
FPS:  3.9010765776038556
